In [36]:
from shapely.geometry import Point, LineString
import geopandas as gp
import os
import pandas as pd
from leaflet_creation import create_map_visualization

from utils import find_files, load_json_file

from split_video_gpx import parse_gpx

In [2]:
def conf2color(conf):
    if conf>0.8:
        return '#1CFF00'
    if conf>0.5:
        return '#4EFFC5'
    if conf>0.1:
        return '#E8AE19'
    return '#FF100F'

In [61]:
from collections import namedtuple
Args = namedtuple('arguments', ['root_dir', 'match_file', 'segs_for_clips_csv', 'segs'])

In [103]:
def concat_gpx_snapped(match_df):
    raw_gpdfs = []
    snapped_gpdfs = []
    for gpx_file, js_file in match_df[match_df.match][['gpx', 'json_file']].values:
        vfile, locs, tms = parse_gpx(gpx_file)
        raw_line = LineString(locs)
        raw_gpdfs.append((raw_line, '#00e', gpx_file, js_file))

        snapped_json = load_json_file(js_file)    
        for bidx, batch in enumerate(snapped_json):
            clip_name = batch['clip_name']
            start, end = batch['start'], batch['end']
            for sbidx, sub_batch in enumerate(batch['snapped']):
                conf = sub_batch['confidence']
                snapped_line = LineString(sub_batch['snapped'])
                snapped_gpdfs.append((snapped_line, conf2color(conf), bidx, sbidx, clip_name, start, end))

    raw_gpdfs = gp.GeoDataFrame(raw_gpdfs, columns=['geometry', 'color', 'gpx_file', 'js_file']).reset_index()
    snapped_gpdfs = gp.GeoDataFrame(snapped_gpdfs, columns=['geometry', 'color', 'batch_id', 'sub_batch_id', 'clip_name', 'start', 'end'])
    return raw_gpdfs, snapped_gpdfs

In [97]:
root_dir = 'test_data/'
match_file = 'gpx_video_match.csv'
segs_for_clips_csv = 'segs_for_clips.csv'
segs = 'segments_dc.geojson'
args = Args(root_dir, match_file, segs_for_clips_csv, segs)

In [14]:
os.chdir(root_dir)

In [98]:
match_df = pd.read_csv(match_file)
segs_for_clips_df = pd.read_csv(segs_for_clips_csv)
segs = gp.read_file(args.segs)

In [99]:
covered_segs = segs_for_clips_df.groupby('index_seg').clip_name.apply(list).apply(lambda x: '<br>'.join(x)).to_frame('clip_name')
covered_segs = segs[['geometry', 'STREETSEGID']].merge(covered_segs, left_index=True, right_index=True)
covered_segs['color'] = '#555'

In [104]:
raw_gpdfs, snapped_gpdfs = concat_gpx_snapped(match_df)

In [105]:
html_title = 'raw_snap'
file_path = ''
file_name = 'raw_snap' 
lon, lat = -77.036484, 38.907215  #D.C.
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets', 'satellite']
gpdfs = [raw_gpdfs, snapped_gpdfs, covered_segs]
binding_data = [['ly1', 'raw traces'], ['ly2', 'snapped traces'], ['ly3', 'covered_segs']]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)

# Abnomal examples

In [191]:
gpx = 'Sample Data/GPX/Track_2017-02-21 125536.gpx'
vfile, raw_locs, tms = parse_gpx(gpx)

snapped_locs = snap2road(raw_locs)

gpdf_raw = gp.GeoDataFrame([Point(x) for x in raw_locs], columns=['geometry']).reset_index()
gpdf_raw.geometry = gpdf_raw.buffer(bfr)
gpdf_raw['color'] = '#aaa'
gpdf_snapped = gp.GeoDataFrame([Point(x) for x in snapped_locs], columns=['geometry']).reset_index()
gpdf_snapped.geometry = gpdf_snapped.buffer(bfr)
gpdf_snapped['color'] = '#800'
gpdfs = [gpdf_raw, gpdf_snapped]
binding_data = [['ly1', 'raw'], ['ly2','snapped']]

html_title = 'anomaly1'
file_path = ''
file_name = 'anomaly1'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets', 'satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)

In [193]:
gpx = 'Sample Data/GPX/Track_2017-02-21 131933.gpx'
vfile, raw_locs, tms = parse_gpx(gpx)

snapped_locs = snap2road(raw_locs)

gpdf_raw = gp.GeoDataFrame([Point(x) for x in raw_locs], columns=['geometry']).reset_index()
gpdf_raw.geometry = gpdf_raw.buffer(bfr)
gpdf_raw['color'] = '#aaa'
gpdf_snapped = gp.GeoDataFrame([Point(x) for x in snapped_locs], columns=['geometry']).reset_index()
gpdf_snapped.geometry = gpdf_snapped.buffer(bfr)
gpdf_snapped['color'] = '#800'
gpdfs = [gpdf_raw, gpdf_snapped]
binding_data = [['ly1', 'raw'], ['ly2','snapped']]

html_title = 'anomaly2'
file_path = ''
file_name = 'anomaly2'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets', 'satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)